#### Day 20 - A

Find number of solutions that save 100 steps if allowed to cheat for 2 steps

In [1]:
#Import Libraries and settings
from copy import deepcopy
import bisect

settings = {
    "day": 20,
    "test_data": 0
}

In [2]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    grid = []
    for idx_y, line in enumerate(lines):
        if "S" in line:
            starting_loc = (line.index("S"), idx_y)
        if "E" in line:
            ending_loc = (line.index("E"), idx_y)
        grid.append(list(line))

    grid_dimens = (len(lines[0]), len(lines))

    return grid, grid_dimens, starting_loc, ending_loc

GRID_BASE, GRID_DIMENSIONS, START_LOC, END_LOC = load_input(settings)

In [3]:
#Convert compass direction into a dir tuple
def translate_step(dir_g, mag=1):
    v = 0
    h = 0

    if "^" in dir_g:
        v = -mag
    elif "v" in dir_g:
        v = mag

    if ">" in dir_g:
        h = mag
    elif "<" in dir_g:
        h = -mag

    return (h, v)

#Apply a direction to a location
def apply_move(loc, dir, mag=1):
    step = translate_step(dir, mag)
    return (loc[0]+step[0], loc[1]+step[1])

#Get all neighbours to consider for a space
def get_neighbours(grid, loc, history, targets=["#"], cheat_max=0):
    dirs = ["^", ">", "v", "<"]
    nbs = []
    base_cost = history[loc]

    for pos_dir in dirs:
        next_loc = apply_move(loc, pos_dir)
        if (
            next_loc[0] >= 0 and 
            next_loc[0] < GRID_DIMENSIONS[0] and
            next_loc[1] >= 0 and
            next_loc[1] < GRID_DIMENSIONS[1]):
            if next_loc not in history.keys():
                next_cost = base_cost + 1
                if (grid[next_loc[1]][next_loc[0]] not in targets) or next_cost <= cheat_max:
                    history[next_loc] = next_cost
                    nbs.append((next_loc, next_cost))

    return nbs

In [4]:
#Algorithm to find the minimum path between S and E
def breadth_first(grid=deepcopy(GRID_BASE), starting_loc=START_LOC, end_loc=END_LOC, cheat=None):

    history = {starting_loc:0}
    queue = [(starting_loc, 0)]

    if cheat is not None:
        grid[cheat[1]][cheat[0]] = "."
    
    while queue:
        next_move = queue[0]
        loc = next_move[0]
        queue.pop(0)

        nbs = get_neighbours(grid, loc, history)
        for future_move in nbs:
            if future_move[0] == end_loc:
                return history, future_move[1]
            bisect.insort(queue, future_move, key=lambda x:x[1])

    return False

In [5]:
#Get the solution without cheating
history, honest_sol = breadth_first()
print(honest_sol)

9380


In [6]:
#Create dictionary with distance from E for each space
cheat_lookup = {}
for key in history.keys():
    cheat_lookup[key] = (history[key], honest_sol-history[key])

In [7]:
#Get all movement deltas from n steps
def get_deltas_for_n(n):
    deltas = set()

    #For each length up to n
    for j in range(1, n+1):
        xy_mags = []
        #Get all moves with positive x and y
        for i in range(j):
            xy_mags.append((i, j-i))
            xy_mags.append((j-i, i))

        #Record a version for positive and negative x and y for different directions
        for mag in xy_mags:
            deltas.add(((mag[0], mag[1]), j))
            deltas.add(((mag[0], -mag[1]), j))
            deltas.add(((-mag[0], mag[1]), j))
            deltas.add(((-mag[0], -mag[1]), j))

    return deltas

#Get all spaces reachable using a set of delta movements
def spaces_for_cheat(loc, deltas, cheat_lookup):
    spaces = []
    for delta in deltas:
        new_loc = (loc[0] + delta[0][0], loc[1] + delta[0][1])
        #Check new location is on the track (Not a wall)
        if new_loc in cheat_lookup.keys():
            spaces.append((new_loc, delta[1]))

    return spaces

In [8]:
#Number of steps to cheat for
cheat_steps = 2
#Count how many solutions that save this many steps
saving_threshold = 100

#Get all deltas for this number of steps (fixed)
deltas = get_deltas_for_n(cheat_steps)

#Check all combination of savings by attempting to cheat from every space on the track to every reachable end point
savings  = 0
for idx, space in enumerate(cheat_lookup.keys()):
    #Get cheat options for this space
    cheat_options = spaces_for_cheat(space, deltas, cheat_lookup)

    #Check how many steps are saved for each cheat option
    for cheat in cheat_options:
        cheat_saving = (cheat_lookup[space][1] - cheat_lookup[cheat[0]][1]) - cheat[1]

        if cheat_saving >= saving_threshold:
            savings += 1

    if idx % 500 == 0:
        print(idx, "processed")

print(savings)

0 processed
500 processed
1000 processed
1500 processed
2000 processed
2500 processed
3000 processed
3500 processed
4000 processed
4500 processed
5000 processed
5500 processed
6000 processed
6500 processed
7000 processed
7500 processed
8000 processed
8500 processed
9000 processed
1381
